### 모델 파인튜닝

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Mount Google Drive (for Colab users)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments, get_scheduler
from datasets import Dataset
import pandas as pd
import torch

# 데이터 불러오기 
file_path = "/content/model1.csv"
data = pd.read_csv(file_path)

# 학습/검증 데이터 분할
split_ratio = 0.8
train_size = int(len(data) * split_ratio)

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:]

def prepare_data(data):
    """
    특정 컬럼(input, output)만 선택해
    Hugging Face Dataset 객체로 변환합니다.
    """
    return Dataset.from_pandas(data[['input', 'output']])

# train, validation용 Dataset 생성
train_dataset = prepare_data(train_data)
val_dataset = prepare_data(val_data)

# KoBART-Summarization 모델 사용
tokenizer = PreTrainedTokenizerFast.from_pretrained("digit82/kobart-summarization")  # gogamza 모델도 사용 가능
model = BartForConditionalGeneration.from_pretrained("digit82/kobart-summarization")

# 데이터 토크나이즈 함수
def tokenize_function(examples):
    """
    입력(input)과 출력(output)을 토크나이즈하고,
    모델 훈련에 필요한 labels까지 함께 반환합니다.
    """
    # 입력 토큰화
    model_inputs = tokenizer(
        examples['input'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    # 출력 토큰화 및 labels 생성
    labels = tokenizer(
        examples['output'],
        max_length=512,
        truncation=True,
        padding="max_length"
    ).input_ids
    model_inputs['labels'] = labels
    return model_inputs

# map 함수를 이용해 전체 dataset에 토크나이즈 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./kobart_results",   # 모델 훈련 결과(체크포인트 등) 저장 경로
    evaluation_strategy="steps",     # 일정 스텝 간격으로 평가를 진행
    eval_steps=500,                  # 500 스텝마다 평가 진행
    save_steps=500,                  # 500 스텝마다 모델 체크포인트 저장
    learning_rate=5e-5,              # 학습률
    per_device_train_batch_size=16,  # 훈련 배치 사이즈
    per_device_eval_batch_size=16,   # 검증 배치 사이즈
    num_train_epochs=30,             # 훈련 에폭 수
    weight_decay=0.01,               # 가중치 감쇠(정규화) 계수
    save_total_limit=2,              # 저장할 체크포인트 수 제한
    logging_dir='./kobart_logs',     # 로그 파일 저장 경로
    logging_steps=50,                # 50 스텝마다 로그 기록
    warmup_steps=100,                # 일정 스텝 동안 학습률 서서히 증가
    fp16=torch.cuda.is_available(),  # GPU 사용 가능 시 FP16(반정밀도) 사용
    load_best_model_at_end=True,     # 학습 종료 시 가장 좋은 모델 로드
    report_to="none"                 # 기본 리포트 툴 비활성화
)

#   옵티마이저 & 스케줄러 설정   #
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

# 훈련에 필요한 전체 스텝 수 계산
num_training_steps = (
    len(train_dataset) * training_args.num_train_epochs // training_args.per_device_train_batch_size
)

# 스케줄러(학습률 조절) 설정
scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=num_training_steps
)

# Early Stopping 콜백 설정
from transformers import EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5) # 5번 연속으로 검증 성능 개선되지 않으면 학습 중단

# Trainer 객체 생성
trainer = Trainer(
    model=model,                       # 훈련에 사용할 모델
    args=training_args,                # 훈련 파라미터
    train_dataset=train_dataset,       # 훈련 데이터셋
    eval_dataset=val_dataset,          # 검증 데이터셋
    tokenizer=tokenizer,               # 토크나이저 (평가 시 디코딩 등에서 사용)
    optimizers=(optimizer, scheduler), # 직접 정의한 옵티마이저, 스케줄러
    callbacks=[early_stopping_callback] # 얼리 스토핑 콜백
)

# 모델 훈련 시작 
trainer.train()

# 모델 저장(결과) 
# 학습 완료 후 최종 모델을 지정된 경로에 저장
trainer.save_model("/content/drive/MyDrive/digit82_batch16_epoch30")


In [ ]:
import re
import json
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 파인튜닝한 모델과 토크나이저 로드
model_path = "/content/drive/MyDrive/digit82_batch16_epoch30"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

def preprocess_and_split(text):
    """
    텍스트 전처리 및 청크 분리 함수
    """
    
    # 불필요한 문자 제거
    text = text.replace('*', '').replace('◆', '').replace('◇', '')
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백 제거
    text = text.replace('\n', ' ').replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")
    text = re.sub(r'\([^가-힣]*[\u4E00-\u9FFF\u3040-\u30FF]+[^가-힣]*\)', '', text)  # 괄호 안 한자/일본어 제거
    text = re.sub(r'[\u4E00-\u9FFF\u3040-\u30FF]+', '', text)  # 한자/일본어 제거
    
    # 10,000자 단위로 청크 분할
    chunks = [text[i:i + 10000] for i in range(0, len(text), 10000)]
    result = []

    # 각 청크를 다시 문장 단위로 분리
    for chunk in chunks:
        buffer = ""
        # 따옴표가 열려 있는지 확인 (문장 중간에서 잘못 끊기는 걸 방지)
        is_open_double_quote = False
        is_open_single_quote = False

        for char in chunk:
            buffer += char

            # 쌍따옴표, 홑따옴표 열림/닫힘 여부
            if char == '"':
                is_open_double_quote = not is_open_double_quote
            elif char == "'":
                is_open_single_quote = not is_open_single_quote

            # 버퍼가 250자 이상이 되면, 문장부호 등을 체크하여 분할 시도
            if len(buffer) >= 250:
                # 마침표, 느낌표, 물음표를 만나고 따옴표가 열려 있지 않다면
                if char in ['.', '!', '?'] and not is_open_double_quote and not is_open_single_quote:
                    result.append(buffer.strip())
                    buffer = ""
                # "" 연속 등장 등 특정 패턴 처리
                elif char in ['"'] and buffer[-3] in ['"']:
                    result.append(buffer[:-1].strip())
                    buffer = '"'
                # '' 연속 등장 등 특정 패턴 처리
                elif char in ["'"] and buffer[-3] in ["'"]:
                    result.append(buffer[:-1].strip())
                    buffer = "'"
                # ... 또는 !!, ?? 같은 연속 문장부호 처리
                elif re.search(r'\.\.\.|!!|\?\?', buffer):
                    match = re.search(r'(\.\.\.|!!|\?\?)', buffer)
                    split_index = match.end()
                    result.append(buffer[:split_index].strip())
                    buffer = buffer[split_index:].strip()
    return result

def parse_output_to_json(generated_output):
    """
    모델 출력을 JSON 형식으로 변환
    """
    parsed_data = {}
    try:
        # [location] 태그 뒤에 오는 내용 추출
        location_match = re.search(r"\[location\]\s*(.+)", generated_output)
        if location_match:
            parsed_data["location"] = location_match.group(1).strip()

        # [caption] 태그 뒤에 오는 내용 추출
        caption_match = re.search(r"\[caption\]\s*(.+)", generated_output)
        if caption_match:
            parsed_data["caption"] = caption_match.group(1).strip()

        # [dialogues] 태그 뒤에 오는 목록 추출
        dialogues_match = re.search(r"\[dialogues\]\s*(\[.+)", generated_output, re.DOTALL)
        if dialogues_match:
            dialogues_raw = dialogues_match.group(1).strip()
            dialogues = []
            # [speaker] / [dialogue] 패턴에 맞춰서 대화 추출
            for speaker, dialogue in re.findall(r"\[speaker\]\s*(.+?)\s*\[dialogue\]\s*(.+?)(?=\[speaker\]|\Z)", dialogues_raw, re.DOTALL):
                dialogue = re.sub(r"\s*\n\s*[\[\]]?", "", dialogue).strip()
                dialogues.append({
                    "speaker": speaker.strip(),
                    "dialogue": dialogue
                })
            parsed_data["dialogues"] = dialogues

    except Exception as e:
        parsed_data["error"] = f"Parsing error: {str(e)}"
        parsed_data["content"] = generated_output

    return parsed_data

def process_file(file_path):
    """
    개별 텍스트 파일을 처리하고 결과를 출력
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        split_text = preprocess_and_split(text) # 전처리 후 청크 분할

        # 청크 단위로 모델 추론 진행
        for idx, chunk in enumerate(split_text):
            print(f"청크 {idx + 1} 처리 중...")

            # 모델 입력 생성
            inputs = tokenizer(chunk, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

            # 모델 출력 생성
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=512,
                num_beams=4,
                length_penalty=1.0,
                early_stopping=True,
            )

            # 모델 출력 디코딩
            generated_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # 디코딩된 문자열을 JSON 형식으로 파싱
            parsed_output = parse_output_to_json(generated_output)

            # JSON 형식으로 출력
            print(f"청크 {idx + 1} 결과:")
            print(json.dumps(parsed_output, indent=4, ensure_ascii=False))  # JSON 형식으로 출력

            # 사용자 확인 후 다음 청크 진행
            user_input = input(f"청크 {idx + 1} 처리를 완료하려면 '완료'를 입력하세요: ")
            if user_input.strip().lower() != "완료":
                print("작업을 중단합니다.")
                return

file_path = r'/content/미로(애장판) 1 (박수정) (Z-Library).txt'
process_file(file_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


청크 1 처리 중...
청크 1 결과:
{
    "location": "\"도시, 클럽 근처\"",
    "caption": "\"윤은 누군가 자신을 따라오고 있다는 것을 느끼며, 그녀가 하룻밤 상대였지만, 그녀가 다음 날부터 여자친구처럼 행동하려는 모습에 당황한다.\"",
    "dialogues": [
        {
            "speaker": "\"윤\"",
            "dialogue": "\"미치겠네.\""
        },
        {
            "speaker": "\"\"",
            "dialogue": "\"\""
        }
    ]
}
청크 1 처리를 완료하려면 '완료'를 입력하세요: 완료
청크 2 처리 중...
청크 2 결과:
{
    "location": "\"도시, 회사 근처\"",
    "caption": "\"윤은 지수의 끊임없는 연락과 협박에 시달리며, 지수의 주장에 헛웃음이 나올 정도로 기가 막혀한다.\"",
    "dialogues": [
        {
            "speaker": "\"\"",
            "dialogue": "\"\""
        },
        {
            "speaker": "\"\"",
            "dialogue": "\"\""
        }
    ]
}
청크 2 처리를 완료하려면 '완료'를 입력하세요: 완료
청크 3 처리 중...
청크 3 결과:
{
    "location": "\"도시, 회사 근처\"",
    "caption": "\"윤은 여대생의 일인시위를 막기 위해 빠르게 상황을 처리하려 한다. 그는 전화를 끊고 나서 다시 연락을 하지 않기로 결심한다. 그러나 상대는 여전히 윤을 뒤쫓고 있다.\"",
    "dialogues": [
        {
            "speaker": "\"\"",
            "dialogue": "\